# 自动微分
:label:`sec_autograd`

回想在 :numref:`sec_calculus` 中提到的，计算导数是训练深度网络所使用的所有优化算法中的关键步骤。虽然这些计算很简单，但手动进行这些计算可能既繁琐又容易出错，而且随着模型变得更加复杂，这些问题只会越来越严重。

幸运的是，所有现代深度学习框架都通过提供*自动微分*（通常简称为*autograd*）来为我们解决这个问题。当我们通过每个连续的函数传递数据时，框架会构建一个*计算图*，跟踪每个值如何依赖于其他值。为了计算导数，自动微分反向遍历这个图并应用链式法则。以这种方式应用链式法则的计算算法被称为*反向传播*。

尽管在过去十年中，自动微分库已成为热门话题，但它们有着悠久的历史。事实上，最早的自动微分参考可以追溯到半个多世纪前 :cite:`Wengert.1964`。现代反向传播背后的核心思想可以追溯到1980年的一篇博士论文 :cite:`Speelpenning.1980`，并在20世纪80年代末得到了进一步发展 :cite:`Griewank.1989`。虽然反向传播已成为计算梯度的默认方法，但它并不是唯一的选择。例如，Julia编程语言采用了前向传播 :cite:`Revels.Lubin.Papamarkou.2016`。在探索方法之前，让我们先掌握自动微分包。

In [1]:
import torch

## 一个简单的函数

假设我们感兴趣于（**对函数
$y = 2\mathbf{x}^{\top}\mathbf{x}$
关于列向量 $\mathbf{x}$ 求导。**）
首先，我们给 `x` 赋一个初始值。

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要一个地方来存储它。**]
通常，我们在每次求导时都避免分配新的内存，
因为深度学习需要
反复对相同的参数
多次计算导数，
这样可能会导致内存耗尽。
请注意，标量值函数关于向量$\mathbf{x}$
的梯度是一个与$\mathbf{x}$形状相同的向量。

In [3]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

（**我们现在计算`x`的函数并将结果赋值给`y`。**）

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

[**我们现在可以通过调用其`backward`方法来计算`y`关于`x`的梯度**]。接下来，我们可以通过`x`的`grad`属性访问该梯度。

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

（**我们已经知道函数 $y = 2\mathbf{x}^{\top}\mathbf{x}$ 对 $\mathbf{x}$ 的梯度应该是 $4\mathbf{x}$。**）
我们现在可以验证自动梯度计算和预期结果是相同的。

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

[**现在我们来计算另一个关于`x`的函数并求其梯度。**]
请注意，当记录新的梯度时，PyTorch不会自动重置梯度缓冲区。
相反，新的梯度会被加到已经存储的梯度上。
当我们想要优化多个目标函数的和时，这种行为非常有用。
要重置梯度缓冲区，我们可以如下调用`x.grad.zero_()`:

In [7]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 向量的反向传播

当 `y` 是一个向量时，
`y` 对向量 `x` 的导数
最自然的表示形式是一个称为*雅可比矩阵*的矩阵，
它包含了 `y` 的每个分量
对 `x` 的每个分量的偏导数。
同样，对于更高阶的 `y` 和 `x`，
求导的结果可能是一个更高阶的张量。

虽然雅可比矩阵确实出现在一些
高级机器学习技术中，
但更常见的是我们希望将
`y` 的每个分量对整个向量 `x` 的梯度
加起来，得到一个与 `x` 形状相同的向量。
例如，我们经常有一个向量
表示损失函数的值
针对一批训练样本中的每个样本分别计算。
这里，我们只是希望（**将为每个样本单独计算的梯度加起来**）。

由于深度学习框架在解释非标量张量的梯度时有所不同，PyTorch采取了一些措施以避免混淆。对非标量调用`backward`会引发错误，除非我们告诉PyTorch如何将对象减少到标量。更正式地说，我们需要提供一个向量$\mathbf{v}$，使得`backward`计算的是$\mathbf{v}^\top \partial_{\mathbf{x}} \mathbf{y}$而不是$\partial_{\mathbf{x}} \mathbf{y}$。接下来的部分可能会让人困惑，但出于稍后会变得清楚的原因，这个参数（代表$\mathbf{v}$）被命名为`gradient`。更多详细描述，请参见Yang Zhang的[Medium文章](https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29)。

In [8]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 分离计算

有时，我们希望[**将某些计算移出记录的计算图之外。**]例如，假设我们使用输入来创建一些辅助中间项，而我们并不想为这些项计算梯度。在这种情况下，我们需要将相应的计算图从最终结果中*分离*出来。下面这个简单的例子可以更清楚地说明这一点：假设我们有`z = x * y`和`y = x * x`，但我们想关注的是`x`对`z`的*直接*影响，而不是通过`y`传递的影响。在这种情况下，我们可以创建一个新的变量`u`，它的值与`y`相同，但其*来源*（它是如何被创建的）已经被清除。因此，`u`在图中没有祖先，梯度不会通过`u`流向`x`。例如，对`z = x * u`求梯度将得到结果`u`（而不是你可能预期的`3 * x * x`，因为`z = x * x * x`）。

In [9]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

请注意，虽然此过程将`y`的祖先节点从通向`z`的计算图中分离出来，但通向`y`的计算图仍然存在，因此我们可以计算`y`相对于`x`的梯度。

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 梯度与Python控制流

到目前为止，我们回顾了从输入到输出的路径通过诸如 `z = x * x * x` 的函数明确定义的情况。
编程为我们提供了更多的自由来计算结果。
例如，我们可以让它们依赖于辅助变量或根据中间结果进行条件选择。
使用自动微分的一个好处是，即使构建函数的计算图需要通过复杂的Python控制流（例如，条件语句、循环和任意函数调用），我们仍然可以计算结果变量的梯度。
为了说明这一点，请考虑以下代码片段，其中 `while` 循环的迭代次数和 `if` 语句的评估都取决于输入 `a` 的值。

In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

在下面，我们调用这个函数，传入一个随机值作为输入。
由于输入是一个随机变量，
我们不知道计算图会采取什么形式。
然而，每当我们对特定输入执行 `f(a)` 时，
我们实现了一个具体的计算图，
并且可以随后运行 `backward`。

In [ ]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

尽管我们的函数 `f` 为了演示的目的有点人为构造，
但它对输入的依赖非常简单：
它是 `a` 的*线性*函数，
并且具有分段定义的比例。
因此，`f(a) / a` 是一个常数项向量，
而且，`f(a) / a` 必须与 `f(a)` 对 `a` 的梯度相匹配。

In [13]:
a.grad == d / a

tensor(True)

动态控制流在深度学习中非常常见。
例如，在处理文本时，计算图
依赖于输入的长度。
在这种情况下，自动微分
对于统计建模变得至关重要，
因为事先计算梯度是不可能的。

## 讨论

你现在应该已经领略到了自动微分的强大之处。
用于计算导数的库
既自动化又高效，
极大地提高了深度学习实践者的生产力，
使他们能够专注于不那么琐碎的任务。
此外，自动微分让我们能够设计庞大的模型，
这些模型如果用笔和纸来计算梯度
将会极其耗时。
有趣的是，虽然我们使用自动微分来*优化*模型
（从统计意义上讲）
自动微分库本身的*优化*
（从计算意义上讲）
是框架设计者非常感兴趣的一个丰富课题。
在这里，编译器和图操作工具被用来
以最便捷和内存效率最高的方式计算结果。

现在，请记住这些基础知识：(i) 将梯度附加到我们希望对其求导的变量上；(ii) 记录目标值的计算；(iii) 执行反向传播函数；(iv) 访问得到的梯度。

## 练习

1. 为什么二阶导数比一阶导数计算成本高得多？
1. 在运行反向传播函数后，立即再次运行它并观察会发生什么。进行调查。
1. 在控制流示例中，我们计算`d`关于`a`的导数，如果我们把变量`a`改为随机向量或矩阵会发生什么？此时，计算`f(a)`的结果不再是标量。结果会怎样变化？如何分析这种情况？
1. 设$f(x) = \sin(x)$。绘制$f$及其导数$f'$的图形。不要利用$f'(x) = \cos(x)$这一事实，而是使用自动微分来获取结果。
1. 设$f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$。画出一个从$x$到$f(x)$的结果依赖图。
1. 使用链式法则计算上述函数的导数$\frac{df}{dx}$，将每个项放在你之前构建的依赖图上。
1. 根据图形和中间导数结果，你在计算梯度时有多种选择。一次是从$x$到$f$开始评估结果，另一次是从$f$回溯到$x$。从$x$到$f$的路径通常被称为*前向微分*，而从$f$到$x$的路径则被称为反向微分。
1. 什么时候你会想使用前向微分，什么时候使用反向微分？提示：考虑所需的中间数据量、步骤并行化的可能性以及涉及的矩阵和向量的大小。

[讨论](https://discuss.d2l.ai/t/35)